# 🚨 Engine Anomaly Detection Notebook

In [ ]:
%pip install scikit-learn

: 

In [ ]:
from normalization_engine import BaseEngine
import numpy as np
from sklearn.ensemble import IsolationForest


: 

In [ ]:
# Select engine type: "petrol", "diesel", or "rotary"
engine = BaseEngine("petrol")
schema = engine.schema


In [ ]:
def generate_normal_data(n=500):
    rng = np.random.default_rng()
    data = []
    for _ in range(n):
        sample = {}
        for key, (min_val, max_val) in schema.items():
            sample[key] = rng.uniform(min_val, max_val)
        norm_sample = engine.normalize(sample)
        data.append(list(norm_sample.values()))
    return np.array(data)

X_train = generate_normal_data()


In [ ]:
model = IsolationForest(contamination=0.01, random_state=42)
model.fit(X_train)


In [ ]:
# Example real-time input
sensor_input = {
    "rpm": 7200,
    "coolant_temp": 98,
    "throttle_pos": 70,
    "pressure": 3.4,
    "vibration": 0.4,
    "air_temp": 35
}

norm_input = list(engine.normalize(sensor_input).values())
prediction = model.predict([norm_input])[0]

status = "ANOMALY ❌" if prediction == -1 else "NORMAL ✅"
print("Input:", sensor_input)
print("Status:", status)


In [ ]:
def label_wise_diagnostics(sensor_input, schema):
    diagnostics = {}
    for key, value in sensor_input.items():
        min_val, max_val = schema.get(key, (0, 1))
        if value < min_val:
            diagnostics[key] = "LOW ⚠️"
        elif value > max_val:
            diagnostics[key] = "HIGH ⚠️"
        else:
            diagnostics[key] = "OK ✅"
    return diagnostics

print("Sensor Diagnostics:")
print(label_wise_diagnostics(sensor_input, schema))
